In [23]:
import pandas as pd
import numpy as np
import pandas as pd
from datetime import datetime as dt
import re
import MeCab
import ipadic
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import scipy as sp
import collections

In [24]:
# xlsxファイルをdfに変換する
df = pd.read_excel("/app/data/patent20230901.xlsx",header=0) 

In [25]:
df.columns

Index(['No', '出願番号', '公開・公表番号', '登録番号', 'ST', 'ステイタス', 'IPC', 'FIファセット付',
       'Ｆターム', '出願人・権利者名', '発明者名', '引用文献情報', '被引用文献情報', '引用文献数', '被引用文献数',
       '発明の名称', '公知日', '出願日', '登録日', '公開・公表日', '原出願日', '筆頭発明者', '筆頭ＩＰＣ', 'FI',
       '早期請求出願有無', '要約', '全請求項', '詳細な説明'],
      dtype='object')

In [26]:
df.head(2)

,No,出願番号,公開・公表番号,登録番号,ST,ステイタス,IPC,FIファセット付,Ｆターム,出願人・権利者名,...,登録日,公開・公表日,原出願日,筆頭発明者,筆頭ＩＰＣ,FI,早期請求出願有無,要約,全請求項,詳細な説明
0,1,特願2023-113165,特開2023-119051,NaN,○,新規出願(経過情報なし),"B66F 9/06 (2006.01),B66F 11/04 (2006.01)","B66F 9/06 P,B66F 11/04",NaN,株式会社大林組,...,NaN,2023.08.25,2020.08.27,富井 孝喜,B66F 9/06 (2006.01),"B66F 9/06 P,B66F 11/04",NaN,(57)【要約】【課題】高所作業の作業効率を向上させることの可能な、高所作業を実施する方法を...,【特許請求の範囲】【請求項１】車両の荷台上に設けられ、高所作業時に展開し移動時に折り畳みの可...,【発明の詳細な説明】【技術分野】【０００１】本発明は、高所作業に用いる移動式足場及び移動式足...
1,2,特願2023-111170,特開2023-118908,NaN,○,新規出願(経過情報なし),"F24F 7/06 (2006.01),F24F 7/007 (2006....","F24F 7/06 C,F24F 7/007 B,F24F 3/16...",NaN,高砂熱学工業株式会社,...,NaN,2023.08.25,2019.03.28,三戸 大介,F24F 7/06 (2006.01),"F24F 7/06 C,F24F 7/007 B,F24F 3/16...",NaN,(57)【要約】【課題】クリーンルームにおけるエネルギーの使用量を少なくする。【解決手段】ク...,【特許請求の範囲】【請求項１】クリーンルームに設置されるクリーンルームシステムであって、前記...,【発明の詳細な説明】【技術分野】【０００１】本発明は、クリーンルームシステム及び空気循環方法...


In [27]:
# drop_list = ['原出願日', '出願番号', '登録番号','ステイタス', 'ST','']
drop_list = ['No', '公開・公表番号', '登録番号', 'ST', 'ステイタス', 'FIファセット付',  '発明者名', '引用文献情報', '被引用文献情報', '引用文献数', '被引用文献数', '発明の名称', '公知日', '出願日', '登録日', '原出願日', '筆頭発明者', '早期請求出願有無']

df = df.drop(drop_list, axis=1)

In [28]:
df.fillna('')

,出願番号,IPC,Ｆターム,出願人・権利者名,公開・公表日,筆頭ＩＰＣ,FI,要約,全請求項,詳細な説明
0,特願2023-113165,"B66F 9/06 (2006.01),B66F 11/04 (2006.01)",,株式会社大林組,2023.08.25,B66F 9/06 (2006.01),"B66F 9/06 P,B66F 11/04",(57)【要約】【課題】高所作業の作業効率を向上させることの可能な、高所作業を実施する方法を...,【特許請求の範囲】【請求項１】車両の荷台上に設けられ、高所作業時に展開し移動時に折り畳みの可...,【発明の詳細な説明】【技術分野】【０００１】本発明は、高所作業に用いる移動式足場及び移動式足...
1,特願2023-111170,"F24F 7/06 (2006.01),F24F 7/007 (2006....",,高砂熱学工業株式会社,2023.08.25,F24F 7/06 (2006.01),"F24F 7/06 C,F24F 7/007 B,F24F 3/16...",(57)【要約】【課題】クリーンルームにおけるエネルギーの使用量を少なくする。【解決手段】ク...,【特許請求の範囲】【請求項１】クリーンルームに設置されるクリーンルームシステムであって、前記...,【発明の詳細な説明】【技術分野】【０００１】本発明は、クリーンルームシステム及び空気循環方法...
2,特願2023-110946,B01D 24/00 (2006.01),,高砂熱学工業株式会社,2023.08.25,B01D 24/00 (2006.01),"B01D 29/08 520C,B01D 29/08 530C,B01D 29/08...",(57)【要約】【課題】フラッシング排水を処理するフィルタ装置の小型化を図る。【解決手段】フ...,【特許請求の範囲】【請求項１】容器内に収容して、配管のフラッシング水を処理する際に使用するフ...,【発明の詳細な説明】【技術分野】【０００１】本発明は、フラッシング水処理用フィルタ、フラッシ...
3,特願2023-109417,G06Q 50/08 (2012.01),5L049CC07,株式会社大林組,2023.08.18,G06Q 50/08 (2012.01),G06Q 50/08,(57)【要約】【課題】揚重作業において、作業現場の安全を確保するための作業支援システム、作...,【特許請求の範囲】【請求項１】三次元空間に構造材モデルを配置するＢＩＭアプリケーションを実行...,【発明の詳細な説明】【技術分野】【０００１】本発明は、クレーン等の揚重装置において、作業支援...
4,特願2023-102503,G06Q 50/08 (2012.01),5L049CC07,株式会社大林組,2023.08.10,G06Q 50/08 (2012.01),G06Q 50/08,(57)【要約】【課題】試験場所とは離れた場所で試験結果を確実かつ迅速に確認、承認できるよう...,【特許請求の範囲】【請求項１】試験実施者が実施した試験の試験結果を、試験管理者に報告する試験...,【発明の詳細な説明】【技術分野】【０００１】本発明は、試験結果報告方法及び試験結果報告システ...
...,...,...,...,...,...,...,...,...,...,...
4791,特願2022-110399,"E02D 29/05 (2006.01),E04G 21/02 (2006....","2D147AB00,2D147AB04,2E172AA05,2E172DA01,2E172D...",株式会社竹中工務店,2022.09.08,E02D 29/05 (2006.01),"E02D 29/05 Z,E02D 29/05 A,E04G 21/02...",(57)【要約】【課題】本発明は、供用中地下構造物の浮き上がりを抑制することを目的とする。【...,【特許請求の範囲】【請求項１】平面視にて供用中地下構造物と隣り合う既設地下構造体を解体し、新...,【発明の詳細な説明】【技術分野】【０００１】本発明は、構造物の施工方法に関する。【背景技術】...
4792,特願2022-110398,E02D 29/05 (2006.01),"2D147AB00,2D147AB04",株式会社竹中工務店,2022.08.26,E02D 29/05 (2006.01),"E02D 29/05 A,E02D 29/05 Z",(57)【要約】【課題】本発明は、供用中地下構造物の浮き上がりを抑制することを目的とする。【...,【特許請求の範囲】【請求項１】平面視にて供用中地下構造物と隣り合う既設地下構造体を解体し、新...,【発明の詳細な説明】【技術分野】【０００１】本発明は、構造物の施工方法に関する。【背景技術】...
4793,特願2021-208648,E02D 29/055 (2006.01),2D147AB04,株式会社竹中工務店,2022.02.14,E02D 29/055 (2006.01),E02D 29/055,(57)【要約】【課題】本発明は、供用中地下構造物の浮き上がりを抑制することを目的とする。【...,【特許請求の範囲】【請求項１】平面視にて供用中地下構造物と隣り合う山留め壁の内側に構造物を施...,【発明の詳細な説明】【技術分野】【０００１】本発明は、逆打ち工法に関する。【背景技術】【００...
4794,特願2021-173966,E21D 11/10 (2006.01),"2D155CA03,2D155DA08,2D155KA00,2D155KC01,2D155K...",株式会社フジタ,2022.01.11,E21D 11/10 (2006.01),E21D 11/10 B,(57)【要約】【課題】省人化、トンネル施工の効率化を図る。【解決手段】コンクリート１６に接...,【特許請求の範囲】【請求項１】覆工コンクリート打設装置に用いるセンサーであって、前記センサー...,【発明の詳細な説明】【技術分野】【０００１】本発明は覆工コンクリート打設装置に用いるセンサー...


In [29]:
# df["company"] = df['出願人・権利者名'].str.split(",")    #出願名をスプリットしてset型に
# # df['datetime'] = [dt.strptime(i.split(',')[0],  '%Y.%m.%d') for i in df['公開・公表日']]  #公開・公表日を datetime64[ns] 型（ Timestamp 型）に 、複数あるのは先頭のだけに
# df['datetime'] = df['公開・公表日'].str.split(",").str[0]  #公開・公表日を datetime64[ns] 型（ Timestamp 型）に 、複数あるのは先頭のだけに
# # df['sentence'] = df['発明の名称'] +df['要約'] + df['全請求項'] .astype("str") + df['詳細な説明']  #なぜか .astype("str") いれないとエラー
# # df['sentence'] = df['発明の名称'] +df['要約'] + df['詳細な説明']  
# # df['sentence'] = df['発明の名称'] +df['要約']        #sentenceの対象にする項目も必要に応じて変える
# df['sentence'] = df['詳細な説明'] + df['要約']      #sentenceの対象にする項目も必要に応じて変える


In [30]:
df["company"] = df['出願人・権利者名'].str.replace(',', '|')
df['f_term'] = df['Ｆターム'].str.replace(',', '|')
df['head_IPC'] = df['筆頭ＩＰＣ'].str[:4]
df['year'] = df['公開・公表日'].str.split(",").str[0].str[:4]
df['sentence'] = df['詳細な説明'] + df['要約']

In [31]:
df.head(2)

,出願番号,IPC,Ｆターム,出願人・権利者名,公開・公表日,筆頭ＩＰＣ,FI,要約,全請求項,詳細な説明,company,f_term,head_IPC,year,sentence
0,特願2023-113165,"B66F 9/06 (2006.01),B66F 11/04 (2006.01)",NaN,株式会社大林組,2023.08.25,B66F 9/06 (2006.01),"B66F 9/06 P,B66F 11/04",(57)【要約】【課題】高所作業の作業効率を向上させることの可能な、高所作業を実施する方法を...,【特許請求の範囲】【請求項１】車両の荷台上に設けられ、高所作業時に展開し移動時に折り畳みの可...,【発明の詳細な説明】【技術分野】【０００１】本発明は、高所作業に用いる移動式足場及び移動式足...,株式会社大林組,NaN,B66F,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、高所作業に用いる移動式足場及び移動式足...
1,特願2023-111170,"F24F 7/06 (2006.01),F24F 7/007 (2006....",NaN,高砂熱学工業株式会社,2023.08.25,F24F 7/06 (2006.01),"F24F 7/06 C,F24F 7/007 B,F24F 3/16...",(57)【要約】【課題】クリーンルームにおけるエネルギーの使用量を少なくする。【解決手段】ク...,【特許請求の範囲】【請求項１】クリーンルームに設置されるクリーンルームシステムであって、前記...,【発明の詳細な説明】【技術分野】【０００１】本発明は、クリーンルームシステム及び空気循環方法...,高砂熱学工業株式会社,NaN,F24F,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、クリーンルームシステム及び空気循環方法...


In [32]:
drop_list2 = ['出願人・権利者名', '公開・公表日', '要約', '全請求項', '詳細な説明', 'FI', '筆頭ＩＰＣ'
              , 'IPC', 'Ｆターム']
df = df.drop(drop_list2, axis=1)

df.head(2)


,出願番号,company,f_term,head_IPC,year,sentence
0,特願2023-113165,株式会社大林組,NaN,B66F,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、高所作業に用いる移動式足場及び移動式足...
1,特願2023-111170,高砂熱学工業株式会社,NaN,F24F,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、クリーンルームシステム及び空気循環方法...


In [33]:
df.head(10)

,出願番号,company,f_term,head_IPC,year,sentence
0,特願2023-113165,株式会社大林組,NaN,B66F,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、高所作業に用いる移動式足場及び移動式足...
1,特願2023-111170,高砂熱学工業株式会社,NaN,F24F,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、クリーンルームシステム及び空気循環方法...
2,特願2023-110946,高砂熱学工業株式会社,NaN,B01D,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、フラッシング水処理用フィルタ、フラッシ...
3,特願2023-109417,株式会社大林組,5L049CC07,G06Q,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、クレーン等の揚重装置において、作業支援...
4,特願2023-102503,株式会社大林組,5L049CC07,G06Q,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、試験結果報告方法及び試験結果報告システ...
5,特願2023-100951,鹿島建設株式会社,NaN,E04H,2023,【発明の詳細な説明】【技術分野】【０００１】本開示は、医療福祉空間に関する。【背景技術】【０...
6,特願2023-97713,株式会社大林組,2E176DD61,E04G,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、建物を解体する場合の破砕物等、建物外部...
7,特願2023-95897,株式会社奥村組|北陸鋼産株式会社|テクノプロ株式会社,2D155BA05|2D155CA04|2D155KA00|2D155KC06,E21D,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、トンネル覆工コンクリートの打設方法に関...
8,特願2023-92620,株式会社カネカ|大成建設株式会社,5F251BA03|5F251BA16|5F251JA02|5F251JA23|5F251JA25,H02S,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、ガラス建材に関する。【背景技術】【００...
9,特願2023-92146,株式会社フジタ,2E164BA46,E04B,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、梁の構造、梁主筋配筋用の支持部材、およ...


In [34]:
# df['sentence'] = df['sentence'].str.replace('\n', '')  #改行コードを削除
# df['sentence'] = df['sentence'].str.replace('\r', '')  #改行コードを削除
# df['company'] = 

In [35]:
df['sentence'] = df['sentence'].astype(str)

In [36]:
excepword = ["位置","部","上記","上部","上端","下記","下部","下端","両端","内部","外部","前記","複数","横","図","要約","方法","手段","対象","現場","評価" ,"可能","構成","当該","以下","以上","該当","項","各","基準","端部","請求", "特許","本","一定","同上", '方向', '程度', '実施例', '実施'] #除外する特定の名詞
exceptype= ["接尾","副詞可能","数詞","助数詞","代名詞","複数"]  #除外する名詞のタイプ


def tokenize_rm_setsubi(sentence):
    """ 連名詞のリストをトークンとして返す """
    # print(".",end='') #動いてるのか不安になるからプリント
    sentence = re.sub('\d+', '', sentence)   # 数字を削除(macabでなぜか名詞に認定された数字あったからここで)
    sentence = re.sub(r'[ -/:-@\[-~]', "。", sentence)#半角記号,数字,英字
    sentence = re.sub(r'[！-／：-＠［-｀｛-～、-〜”’%]+/g', "。", sentence)#全角記号
    sentence = re.sub(r'[[０-９ａ-ｚＡ-Ｚ]]', '', sentence) #全角英数を削除
    
    
    wakati = MeCab.Tagger(ipadic.MECAB_ARGS)
    node = wakati.parseToNode(sentence)
    sequence = 2   #何連名詞以上をかえすか
    dictionary = []
    prev_seq = 0  #その前にいくつ名詞続いたか
    flag = 0  #
    while node:
        word = node.surface
        hinshi = node.feature.split(",")

        if prev_seq  : # その前に名詞が1以上つづいていたら
            if  (hinshi[0] == "名詞" ) and  (hinshi[0] not in exceptype) and  (hinshi[1] not in exceptype )and (hinshi[2] not in exceptype )and (word not in excepword) : #今回、除外名詞以外だったら、
                dictionary[-1] = dictionary[-1]+word   # 辞書の最後の単語にこの単語を結合
                prev_seq  +=1
                flag = 0
                
            elif (hinshi[1] == "接尾" ): #接尾辞だったら
                dictionary[-1] = dictionary[-1]+word   # とりあえず辞書の最後の単語にこの単語を結合して
                prev_seq  +=1
                flag = 1 #フラグを1に

            elif flag == 1: # 一個前が接尾辞で今度が名詞でなかったら、一個前の接尾辞で終わってる単語を辞書から削除
                dictionary.pop(-1)   #一個前の接尾辞で終わってる単語を辞書から削除
                prev_seq  =0
                flag = 0
            else:
                flag = 0
                if prev_seq  < sequence: 
                    dictionary.pop(-1) #一個前の単語を辞書から削除
                prev_seq = 0
        else:  # その前に名詞がつづいてなくて
            if  (hinshi[0] == "名詞" or hinshi[0] == "接頭辞")  and  hinshi[0] not in exceptype and  hinshi[1] not in exceptype and hinshi[2] not in exceptype and word not in excepword :
                dictionary.append(word)
                prev_seq += 1
                flag = 0

        node = node.next  
    return dictionary

In [37]:
# 単語ー文書行列の作成
docs = np.array(df['sentence'])

tfidf_vectorizer = TfidfVectorizer(
    tokenizer = tokenize_rm_setsubi, #上でつくった関数
    lowercase=True,
    max_df=5000,
    # smooth_idf = False,
    min_df=5)

tfidf_vectorizer.fit(docs)
terms= tfidf_vectorizer.get_feature_names()

count_vectrizer = CountVectorizer(
    vocabulary = terms,
    tokenizer = tokenize_rm_setsubi, #上でつくった関数
)

c_matrix = count_vectrizer.transform(docs)

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [38]:
# np.save('c_matrix_allterm',c_matrix)
sp.sparse.save_npz('/app/data/c_matrix_allterm2023',c_matrix)
terms_df = pd.DataFrame(terms)
terms_df.to_csv('/app/data/terms_df2023.csv', mode='w')

# c_matrix = np.load('c_matrix_allterm.npz', allow_pickle=True)
# terms_df = pd.read_csv('terms_df.csv')


In [39]:
c_df = pd.DataFrame(terms_df.values, columns=["terms"] )
c_df["document_frequency"] =  np.array(c_matrix.astype(bool).sum(axis=0).reshape(-1,1))
c_df["max_term_frequency"]  = c_matrix.T.max(axis=1).toarray()
c_df["sumof_term_frequency"] = np.array(c_matrix.T.sum(axis=1))
c_df.sort_values('document_frequency', ascending=False) # 場合によってはコメントアウト

,terms,document_frequency,max_term_frequency,sumof_term_frequency
5374,技術分野,4703,3,4761
8756,背景技術,4693,5,4789
2637,先行技術文献,4659,3,4668
7689,特開,3532,14,5278
4319,変形例,1202,89,6784
...,...,...,...,...
8104,研掃開始,5,19,38
8107,研究成果,5,1,5
2695,入力インタフェース,5,3,7
4554,学習効率,5,4,12


In [40]:
#単語:文書内単語頻度 の辞書の列つくって、pandasにくっつける
def makeTermsTfDict(terms, vec):
    dic = {key: val for key, val in zip(terms, vec)  if val } #dfが1より大きいtermの辞書つくる
    # dic = {key: val for key, val in zip(terms, vec)  }
    # 20回以上登場する単語のみ抽出
    dic = {key: val for key, val in zip(terms, vec)  if val > 20 }
    term = ''
    for key in dic.keys():
        term += key + '|'
    # 最後の|を削除
    term = term[:-1]
    # dic2 = dict(sorted(dic.items(), key=lambda x:x[1],reverse=True)) #dfの値でソート
    return term

df['terms'] = pd.Series([makeTermsTfDict(terms, vec) for vec in [c_matrix.getrow(i).toarray()[0] for i in range(c_matrix.shape[0])]]) #countされたベクトルの各行を関数に入れていってシリーズに

df.to_excel('/app/data/patent2023ver.xlsx')
# df.to_json('parcedPatents.json')

In [41]:
df.head(10)

,出願番号,company,f_term,head_IPC,year,sentence,terms
0,特願2023-113165,株式会社大林組,NaN,B66F,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、高所作業に用いる移動式足場及び移動式足...,高所作業
1,特願2023-111170,高砂熱学工業株式会社,NaN,F24F,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、クリーンルームシステム及び空気循環方法...,クリーンルーム|環境条件
2,特願2023-110946,高砂熱学工業株式会社,NaN,B01D,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、フラッシング水処理用フィルタ、フラッシ...,保持部材|粒径
3,特願2023-109417,株式会社大林組,5L049CC07,G06Q,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、クレーン等の揚重装置において、作業支援...,支援装置
4,特願2023-102503,株式会社大林組,5L049CC07,G06Q,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、試験結果報告方法及び試験結果報告システ...,測定データ|端末装置|管理装置|試験項目
5,特願2023-100951,鹿島建設株式会社,NaN,E04H,2023,【発明の詳細な説明】【技術分野】【０００１】本開示は、医療福祉空間に関する。【背景技術】【０...,エレベーターホール
6,特願2023-97713,株式会社大林組,2E176DD61,E04G,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、建物を解体する場合の破砕物等、建物外部...,支柱，|防護ネット
7,特願2023-95897,株式会社奥村組|北陸鋼産株式会社|テクノプロ株式会社,2D155BA05|2D155CA04|2D155KA00|2D155KC06,E21D,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、トンネル覆工コンクリートの打設方法に関...,アーチ形状部分|コンクリートポンプ|圧送配管|工コンクリート|工空間
8,特願2023-92620,株式会社カネカ|大成建設株式会社,5F251BA03|5F251BA16|5F251JA02|5F251JA23|5F251JA25,H02S,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、ガラス建材に関する。【背景技術】【００...,ガラス基板|太陽電池
9,特願2023-92146,株式会社フジタ,2E164BA46,E04B,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、梁の構造、梁主筋配筋用の支持部材、およ...,支持部材|梁主筋


In [42]:
df.T.to_json('/app/data/patent2023ver.json', force_ascii=False, indent=4)

In [43]:
import pickle
with open('/app/data/patent_df2023.pkl', 'wb') as f:
    pickle.dump(df, f)

In [44]:
df

,出願番号,company,f_term,head_IPC,year,sentence,terms
0,特願2023-113165,株式会社大林組,NaN,B66F,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、高所作業に用いる移動式足場及び移動式足...,高所作業
1,特願2023-111170,高砂熱学工業株式会社,NaN,F24F,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、クリーンルームシステム及び空気循環方法...,クリーンルーム|環境条件
2,特願2023-110946,高砂熱学工業株式会社,NaN,B01D,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、フラッシング水処理用フィルタ、フラッシ...,保持部材|粒径
3,特願2023-109417,株式会社大林組,5L049CC07,G06Q,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、クレーン等の揚重装置において、作業支援...,支援装置
4,特願2023-102503,株式会社大林組,5L049CC07,G06Q,2023,【発明の詳細な説明】【技術分野】【０００１】本発明は、試験結果報告方法及び試験結果報告システ...,測定データ|端末装置|管理装置|試験項目
...,...,...,...,...,...,...,...
4791,特願2022-110399,株式会社竹中工務店,2D147AB00|2D147AB04|2E172AA05|2E172DA01|2E172D...,E02D,2022,【発明の詳細な説明】【技術分野】【０００１】本発明は、構造物の施工方法に関する。【背景技術】...,掘削領域
4792,特願2022-110398,株式会社竹中工務店,2D147AB00|2D147AB04,E02D,2022,【発明の詳細な説明】【技術分野】【０００１】本発明は、構造物の施工方法に関する。【背景技術】...,掘削領域|施工工程
4793,特願2021-208648,株式会社竹中工務店,2D147AB04,E02D,2022,【発明の詳細な説明】【技術分野】【０００１】本発明は、逆打ち工法に関する。【背景技術】【００...,山留め壁|掘削領域
4794,特願2021-173966,株式会社フジタ,2D155CA03|2D155DA08|2D155KA00|2D155KC01|2D155K...,E21D,2022,【発明の詳細な説明】【技術分野】【０００１】本発明は覆工コンクリート打設装置に用いるセンサー...,レベルセンサー|型枠本体
